In [1]:
import pylast
import pandas as pd
from pathlib import Path
import requests
from IPython.display import Image, display
import ast
import os
from glob import glob
from tqdm.auto import tqdm

In [9]:
tqdm.pandas()

/home/zach/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
API_KEY = "e22d3ef6eb85b8b554fc0a7e3832cd75"  # this is a sample key
API_SECRET = "7e575d3a57cb58e19ebbba4dbee919db"

# In order to perform a write operation you need to authenticate yourself
username = "zacharyneveu"
password_hash = "817ff0e30b8425e4bb511be5969a7115"

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                               username=username, password_hash=password_hash)

In [19]:
datap = Path("/home/zach/Data/fma_medium/fma_meta")

In [20]:
df = pd.read_csv(datap/"raw_tracks.csv")

In [21]:
df.head()

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...


In [22]:
def fix_genres(genre_string):
    """ Turns strings in genre dataframe into a single-genre dictionary
    """
    try:
        return ast.literal_eval(genre_string)[0]
    except Exception as E:
        return None

In [23]:
df['genre'] = df.apply(lambda row: fix_genres(row.track_genres), axis=1)

In [34]:
print(glob("covers/Electronic/Taitsu*"))

['covers/Electronic/Taitsu [Tights].png']


In [40]:
def sanitize_fn(fn):
    """ Sanitizes file names (remove [] and / so they work with globs)
    """
    for ch in ['[', ']']:
        if ch in fn:
            fn = fn.replace(ch, '')
    return fn

In [ ]:
for loc,row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        if row.genre is None:
            continue
        genre = row.genre['genre_title']
        artist, title = str(row.artist_name), str(row.album_title)
        os.makedirs("covers/"+genre, exist_ok=True)
        fn = sanitize_fn("covers/"+genre+"/"+title.replace('/', '_'))
        if len(glob(fn+"*")) > 0:
            # skip re-downloading covers
            continue
        album = network.get_album(artist, title)
        try:
            cover = album.get_cover_image()
            if cover is not None:
                img = requests.get(cover)
                fn += '.'+cover.split('.')[-1]
                with open(fn, 'wb') as fh:
                    fh.write(img.content)
                display(Image(filename=fn))
            else:
                Path(fn+".txt").touch()
                #print(f"Cover not found for {title} by {artist}")
        except Exception as E:
            if E is KeyboardInterrupt:
                break
            else:
                Path(fn+".txt").touch()
                #print(f"{title} not found")
    except Exception as E:
        if E is KeyboardInterrupt:
            break
        else:
            print(loc)
            print(E)

In [ ]:
!rm -r *.txt

In [41]:
len(glob("covers/Metal/Black Elk*"))

1